In [1]:
import os
import sys

sys.path.append("..")

import critical_loads as cl
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import rioxarray as rio
import xarray as xr
from osgeo import gdal

# Exceedances for vegetation and water using the new deposition grid from Met.no

The new workflow was originally designed to accept deposition data in a standard format and spatial resolution. Unfortunately, the data formats and resolution seem to change each time. The workflow currently supports three different grid definitions (BLR, EMEP and 0.1 degree), and *ad hoc* scripts are used to process the various data formats. 

For the most recent data submission (2017 to 2021), we have *two* new data formats and resolutions: a set of 1 km grids from NILU ("old method"; supplied as raw numpy arrays) and a set of 2.5 km grids from Met.no ("new method"; supplied as CSVs). **Continuing to extend the workflow to handle all these variants is not sustainable**. This notebook therefore attempts to "hack" the exceedance calculations for water (SSWC and FAB) and vegetation using the Met deposition data.

In order to be compatible with the rest of the workflow, we need to generate a 120 m resolution version of the Met.no dataset, snapped to the 120 m BLR mask.

## 1. Process Met.no deposition data

In [2]:
# User options
cell_size = 120
ndv = -9999
short_name = "1721_metgrid"
ser_id = -1  # Not actually a series in the db

met_nc = r"/home/jovyan/shared/critical_loads/raster/deposition/from_met_2023/met_dep_2017-21_merged.nc"
mask_tif = f"/home/jovyan/shared/critical_loads/raster/blr_land_mask_{cell_size}m.tif"

# Read data
dep_ds = rio.open_rasterio(met_nc).sel(band=1)
mask_ds = rio.open_rasterio(mask_tif).sel(band=1)

# Aggregate dep data
dep_ds["ndep_mgpm2pyr"] = dep_ds["no3"] + dep_ds["nh4"]
dep_ds["sdep_mgpm2pyr"] = dep_ds["xso4"]
del dep_ds["xso4"], dep_ds["nh4"], dep_ds["no3"]

# Resample and reproject
dep_ds = dep_ds.rio.reproject_match(mask_ds)

# Write to .tif
for par in ["ndep_mgpm2pyr", "sdep_mgpm2pyr"]:
    out_path = f"/home/jovyan/shared/critical_loads/raster/deposition/{par}_{short_name}_{cell_size}m.tif"
    da = dep_ds[par]
    # Only values where the land mask is 1 and the grid is not NoData
    da = da.where((da != da._FillValue) & (mask_ds == 1))
    da = da.fillna(ndv)#.round(0)
    da.rio.write_nodata(ndv, inplace=True)
    da.rio.to_raster(out_path, compress="lzw", dtype="float32")

## 2. SSWC

In [3]:
cl_fold = r"/home/jovyan/shared/critical_loads/raster/water/1721_old"  # This is the same regardless of the dep series, so can use any of the '1721' folders
ex_fold = r"/home/jovyan/shared/critical_loads/raster/exceedance/1721_metgrid"
ex_df = cl.calculate_water_exceedance_sswc(
    ser_id,
    short_name,
    cl_fold,
    ex_fold,
    cell_size=cell_size,
    bc0="BC0",
    neg_to_zero=False,
)
ex_df["short_name"] = short_name
ex_df

Exceedance grid saved to:
    /home/jovyan/shared/critical_loads/raster/exceedance/1721_metgrid/sswc_ex_meqpm2pyr_1721_metgrid_120m.tif


,series_id,medium,total_area_km2,exceeded_area_km2,exceeded_area_pct,short_name
0,-1,water_sswc,322184,20306,6,1721_metgrid


## 3. FAB

In [4]:
# Snap tiff
snap_tif = f"/home/jovyan/shared/critical_loads/raster/blr_land_mask_{cell_size}m.tif"
base_path = r"/home/jovyan/shared/critical_loads/raster"

# Read CL arrays
array_dict = {}
for par in [
    "clminn_meqpm2pyr",
    "clmaxnoaa_meqpm2pyr",
    "clmins_meqpm2pyr",
    "clmaxsoaa_meqpm2pyr",
]:
    # Read tif
    tif_path = os.path.join(base_path, "water", "1721_old", f"{par}_{cell_size}m.tif")
    data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)

    # Set NDV
    data[data == ndv] = np.nan

    # Add to dict
    array_dict[par] = data

# Read dep arrays
for par in ["ndep_mgpm2pyr", "sdep_mgpm2pyr"]:
    # Read tif
    tif_path = os.path.join(
        base_path, "deposition", f"{par}_{short_name}_{cell_size}m.tif"
    )
    data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)
    data = data.astype(np.float32)

    # Set NDV
    data[data == ndv] = np.nan

    # Add to dict
    array_dict[par] = data

# Extract arrays from dict
cln_min = array_dict["clminn_meqpm2pyr"]
cln_max = array_dict["clmaxnoaa_meqpm2pyr"]
cls_min = array_dict["clmins_meqpm2pyr"]
cls_max = array_dict["clmaxsoaa_meqpm2pyr"]
dep_n = array_dict["ndep_mgpm2pyr"] / 14  # Convert to meq
dep_s = array_dict["sdep_mgpm2pyr"] * 2 / 32.06  # Convert to meq

# Estimate exceedances
ex_n, ex_s, reg_id = cl.vectorised_exceed_ns_icpm(
    cln_min, cln_max, cls_min, cls_max, dep_n, dep_s
)

# Get exceeded area
ex = ex_n + ex_s
ex_area = np.count_nonzero(ex > 0) * cell_size * cell_size / 1.0e6
nor_area = np.count_nonzero(~np.isnan(dep_s)) * cell_size * cell_size / 1.0e6
ex_pct = 100 * ex_area / nor_area

# Build df and tidy
ex_df = pd.DataFrame(
    {
        "exceeded_area_km2": ex_area,
        "total_area_km2": nor_area,
        "exceeded_area_pct": ex_pct,
    },
    index=[0],
)

ex_df = ex_df.round(0).astype(int)
ex_df["series_id"] = ser_id
ex_df["medium"] = "water_fab"

ex_df = ex_df[
    [
        "series_id",
        "medium",
        "total_area_km2",
        "exceeded_area_km2",
        "exceeded_area_pct",
    ]
]

ex_df["short_name"] = short_name

# Save results
# N
n_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/{short_name}/fab_exn_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_n, n_tif, snap_tif, -1, gdal.GDT_Float32)

# S
s_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/{short_name}/fab_exs_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_s, s_tif, snap_tif, -1, gdal.GDT_Float32)

# N+S
ns_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/{short_name}/fab_exns_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_n + ex_s, ns_tif, snap_tif, -1, gdal.GDT_Float32)

# Exceedance 'region'
reg_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/{short_name}/fab_ex_reg_id_{short_name}_{cell_size}m.tif"
cl.write_geotiff(reg_id, reg_tif, snap_tif, -1, gdal.GDT_Float32)

ex_df

,series_id,medium,total_area_km2,exceeded_area_km2,exceeded_area_pct,short_name
0,-1,water_fab,322184,35984,11,1721_metgrid


## 4. Vegetation

In [5]:
cl_tif = f"/home/jovyan/shared/critical_loads/raster/sat_veg_{cell_size}m_cl_100smgn_m2_yr_subdiv_water.tif"
ndep_tif = f"/home/jovyan/shared/critical_loads/raster/deposition/ndep_mgpm2pyr_{short_name}_{cell_size}m.tif"
ex_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/{short_name}/veg_ex_mgpm2pyr_{short_name}_{cell_size}m.tif"
ex_tif_bool = f"/home/jovyan/shared/critical_loads/raster/exceedance/{short_name}/veg_ex_{short_name}_{cell_size}m_bool.tif"
ex_df = cl.calc_vegetation_exceedance(ndep_tif, cl_tif, ex_tif, ex_tif_bool, ser_id)
ex_df["short_name"] = short_name
ex_df

,series_id,medium,total_area_km2,exceeded_area_km2,exceeded_area_pct,short_name
0,-1,vegetation,322184,46818,15,1721_metgrid
